In [1]:
from __future__ import absolute_import, division

import os
import urllib, cStringIO

import pymongo as pm

import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import sys

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

### setting up paths


In [2]:
# directory & file hierarchy
proj_dir = os.path.abspath('../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
features_dir= os.path.join(results_dir,'features')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))

if not os.path.exists(results_dir):
    os.makedirs(results_dir)  

if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)  

if not os.path.exists(features_dir):
    os.makedirs(features_dir)
    
## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))        
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

<module 'analysis_helpers' from '/Users/judithfan/semantic_parts/analysis/analysis_helpers.pyc'>

#### helpers

In [14]:
def cleanup_df(X):
    if 'Unnamed: 0' in X.columns:
        X = X.drop(columns=['Unnamed: 0'])
    return X

#### load in data

In [15]:
## loading in files we need
feature_vec = np.load(os.path.join(features_dir,'semantic_parts_sketch_features.npy'))
feature_df = cleanup_df(pd.read_csv(os.path.join(csv_dir,'semantic_parts_sketch_features_key.csv')))
meta_df = cleanup_df(pd.read_csv(os.path.join(csv_dir,'semantic_parts_sketch_features_meta.csv')))

In [30]:
## concatenate meta and features to enable easy subsetting of dataframe
F = pd.concat((meta_df,feature_df),axis=1)

In [ ]:
## aggregate by target and condition and take the mean across rows within each group
F2 = F.groupby(['target','condition']).mean().reset_index()

In [60]:
## construct dictionary of object to part labels

## version where each part label within category is treated distinctly
part_dict_expanded = {'bird': ['beak', 'body', 'eye', 'feet', 'head', 'leg', 'tail', 'wing'], \
                     'car': ['body.1', 'bumper', 'door', 'headlight', 'hood', 'trunk', 'wheel', 'window', 'windshield'],\
                     'chair': ['armrest', 'backrest', 'leg.1', 'seat'], \
                     'dog': ['body.2', 'ear', 'eye.1', 'head.1', 'leg.2', 'mouth', 'neck', 'paw', 'tail.1']}

## version where each repeating part labels within category are not distinguished
part_dict_compressed = {'bird': ['beak', 'body', 'eye', 'feet', 'head', 'leg', 'tail', 'wing'], \
                     'car': ['body', 'bumper', 'door', 'headlight', 'hood', 'trunk', 'wheel', 'window', 'windshield'],\
                     'chair': ['armrest', 'backrest', 'leg', 'seat'], \
                     'dog': ['body', 'ear', 'eye', 'head', 'leg', 'mouth', 'neck', 'paw', 'tail']}

## abbreviated handles 
pde = part_dict_expanded
pdc = part_dict_compressed